**Dependencies** (all of these packages should be added to the python path in the .bashrc

Python 2 (several of the packages below are not compatible yet with python 3)

Healpy (from pip installer)

Flipper (from https://github.com/sudeepdas/flipper) {this needs PyFits and astropy installed via pip}

FlipperPol (from https://github.com/msyriac/flipperPol)

PolSpice (from http://www2.iap.fr/users/hivon/software/PolSpice/)


In [2]:
import flipperPol as fp

ImportError: No module named flipperPol